## Attention 신경망 구현 및 학습

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pwd

'/content'

In [0]:
path = '/content/drive/My Drive/Colab Notebooks'

In [0]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 162kB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 2.4MB 43.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [0]:
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tqdm.notebook import tqdm

## 하이퍼 파라미터

In [0]:
EPOCHS = 50
NUM_WORDS = 10000

## Encoder

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        H, h, c = self.lstm(x)
        return H, h, c

## Decoder

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.att = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, s0, c0, H = inputs
        x = self.emb(x)
        S, h, c = self.lstm(x, initial_state=[s0, c0])
        
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

## Seq2seq

In [0]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs
            H, h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c, H))
            return y
        else:
            x = inputs
            H, h, c = self.enc(x)
            
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c, H])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [0]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [0]:
pwd

'/content'

In [0]:
dataset_file = path + '/ChatbotData.csv'
data = pd.read_csv(dataset_file, encoding='utf-8')

In [0]:
data = data.reindex(np.random.permutation(len(data)))
data

,Q,A,label
232,공부하는 이유?,공부하면 더 많은 선택을 할 수 있죠.,0
3524,우산이 없는데 빌릴까,한번 말해보세요.,0
2895,아이라이너 문신 해볼까,화장하는 시간이 줄어들 거예요.,0
3349,오늘은 별도 안 보이네,한적한 시골에서 하늘을 올려봐보세요.,0
4571,치과 가면 돈 많아 들텐데,나중에 가면 돈이 더 많이 나갈거예요.,0
...,...,...,...
2252,뻘짓 한 느낌 싫다,바람 좀 쐬고 오시면 좋은텐데.,0
3988,잠을 잘못 잤나,뻐근한가봐요.,0
6488,보고싶다는 생각이 너무 간절하네,이별이 만들어낸 허상일 수도 있어요.,1
5654,그녀의 카톡이 없어졌다.,마음에서도 놓아주세요.,1


In [0]:
questions = [lines + ' \n' for lines in list(data['Q'])][:5000]
answers = ['\t ' + lines + ' \n' for lines in list(data['A'])][:5000]

print(questions)
print(answers)
num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)



tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a + test_q + test_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)


['공부하는 이유? \n', '우산이 없는데 빌릴까 \n', '아이라이너 문신 해볼까 \n', '오늘은 별도 안 보이네 \n', '치과 가면 돈 많아 들텐데 \n', '섹스리스 커플이야 \n', '넌 그냥 외로움을 달래기 위한 거였겠지. \n', '삼겹살 냄새 \n', '6개월이 지나도 왜이런거죠? \n', '헤어지누여자친구 만나러가ㅠ \n', '시간표대로 움직여 \n', '일은 많이 하는데 월급은 너무 적어 \n', '인센티브가 생각보다 안 나올 것 같아 \n', '이별했네 \n', '이제 더이상 남자친구로 안느껴진다는 그녀 \n', '여자친구가 화나면 말을 안 해. \n', '비오는날 뭐해? \n', '오늘도 평온해 \n', '그동안 보내왔던 시간. \n', '사람들이 날 너무 힘들게 한다. \n', '눈썹 문신 하면 편할까? \n', '상처가 많은 사람을 사랑하는 것은 힘든 일이었어. \n', '오늘 초미세먼지도 심하대 \n', '꿈에 나왔네 \n', '허전하다 \n', '용돈 인상해 달라고 말할까 \n', '이제 청첩장 돌려도 되겠지? \n', '너무 설레서 울렁거려. \n', '결혼하는데 돈 얼마나 들까 \n', '외로워 \n', '위로 좀 해줘 \n', '좋아하는 오빠랑 친해지는법 \n', '남편이 게이였어 \n', '재밌는거 할 줄 알아 \n', '면접 준비 방법 \n', '지금이라도 중국어 공부할까? \n', '친구 사이로 지낸다는 게 말이 되나 \n', '여친이 있는 사람인데 왜 나한테 잘해주지? \n', '자격증 딸거야 \n', '그는 내가 좋아하는 마음을 이용하는 걸까? \n', '그동안 감정적으로 너무 기대한 듯 \n', '밥 뭐 먹을까 \n', '정신 차려야 하는데 \n', '남동생한테 자꾸 화내게 되네 \n', '서로 위로하며 함께 극복할 사람 \n', '스키 첨 타 봤어. \n', '공부 시작해도 될까 \n', '달달한 거 먹고 싶다. \n', '오늘도잠을못이루네 \n', '스타벅스 같은데 가봤어? \n', '2년 가까이 여친하구 헤

In [0]:
print(test_q)
print(test_a)

['아이라이너 문신 해볼까 \n', '인센티브가 생각보다 안 나올 것 같아 \n', '여자친구가 화나면 말을 안 해. \n', '그동안 보내왔던 시간. \n', '상처가 많은 사람을 사랑하는 것은 힘든 일이었어. \n', '외로워 \n', '남편이 게이였어 \n', '지금이라도 중국어 공부할까? \n', '여친이 있는 사람인데 왜 나한테 잘해주지? \n', '남동생한테 자꾸 화내게 되네 \n', '오늘도잠을못이루네 \n', '스타벅스 같은데 가봤어? \n', '남자친구가 잘생겨서 인기가 거의 아이돌급이야 \n', '정말 서로 좋았는데 엊갈림. \n', '그녀는 지금 뭐하고 있을까? \n', '청첩장 누구한테 줘야할지 \n', '학교 가면 볼텐데 \n', '약속 끝나고 짝녀를 집에 데려다주면 너무 티나려나? \n', '잘지내나 \n', '완전히 끝나버린 사이? \n', '와 연락하고싶다. \n', '연애의 끝은 결혼인거야? \n', '뉴게임 없나? \n', '내가 사랑했던 사람 \n', '서류도 안돼 \n', '발 아파 \n', '바보같이 연락했는데 읽씹당했어 \n', '영어학원 다녀볼까 \n', '양배추즙 맛없어 \n', '이별한지 1달 \n', '어떤 사람을 만나는 게 좋을까? \n', '오늘 연락왔네 그래도 이젠 안녕 \n', '불 날 뻔했어 \n', '새신발 신고 놀러 가야지 \n', '이별후 한달째 인데 \n', '소개팅 하고싶다 \n', '입소문 난 맛집 다녀왔어 \n', '슬픔활용법. \n', '짝사랑 그만하고 싶다 \n', '이별. 냉정해진다 \n', '자꾸 화가나 \n', '친구 예전 남자친구 사귀어도 되나 \n', '허리가 계속 아파. \n', '찜질방 같이 가면 화장 지워야 할까 \n', '내 마음을 보여주는 게 어려워 \n', '죽을 때까지 함께하고 싶다 \n', '나이 차이 많이 나는 짝녀에게 고백해도될까? \n', '이러고있는게 잘하고있는건지. \n', '사랑할수록 아프다 \n', '자기 잘못 인정 안하는 사람 \n', '놀이공원 데이트

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [0]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [0]:
for epoch in tqdm(range(EPOCHS)):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 0.9140341877937317, Accuracy: 92.27696990966797
Epoch 2, Loss: 0.49416491389274597, Accuracy: 94.00514221191406
Epoch 3, Loss: 0.4528266191482544, Accuracy: 94.38726806640625
Epoch 4, Loss: 0.42836031317710876, Accuracy: 94.44548797607422
Epoch 5, Loss: 0.41429999470710754, Accuracy: 94.50878143310547
Epoch 6, Loss: 0.4040398895740509, Accuracy: 94.56192016601562
Epoch 7, Loss: 0.3939703702926636, Accuracy: 94.58302307128906
Epoch 8, Loss: 0.38362500071525574, Accuracy: 94.61740112304688
Epoch 9, Loss: 0.37258079648017883, Accuracy: 94.65882110595703
Epoch 10, Loss: 0.36053580045700073, Accuracy: 94.711181640625
Epoch 11, Loss: 0.34769025444984436, Accuracy: 94.78346252441406
Epoch 12, Loss: 0.3344011604785919, Accuracy: 94.86473083496094
Epoch 13, Loss: 0.32009434700012207, Accuracy: 94.94678497314453
Epoch 14, Loss: 0.3053020238876343, Accuracy: 95.05619049072266
Epoch 15, Loss: 0.2898801863193512, Accuracy: 95.18238830566406
Epoch 16, Loss: 0.2739642858505249, Accurac

## 테스트 루프

In [0]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('질문: ', test_text)
    print('정답: ', gt_text)
    print('예상답: ', texts)

_
질문:  ['아이라이너 문신 해볼까 \n']
정답:  ['\t 화장하는 시간이 줄어들 거예요 \n']
예상답:  ['상대방에게 필요한 실용적인 수 없어요 \n']
_
질문:  ['생각보다 안 것 같아 \n']
정답:  ['\t 것보다 낫죠 \n']
예상답:  ['새로운 스타일 믿어보세요 \n']
_
질문:  ['여자친구가 말을 안 해 \n']
정답:  ['\t 시간을 주세요 \n']
예상답:  ['직접적으로 물어보는 게 가장 필요하죠 \n']
_
질문:  ['그동안 시간 \n']
정답:  ['\t 바랄게요 \n']
예상답:  ['사랑하는 사람의 연락이었나봐요 \n']
_
질문:  ['상처가 많은 사람을 사랑하는 것은 힘든 \n']
정답:  ['\t 마음을 자신의 마음이 하니까요 \n']
예상답:  ['사랑은 변화가 좀 할 것 같아요 \n']
_
질문:  ['외로워 \n']
정답:  ['\t 혼자가 아니에요 \n']
예상답:  ['혼자가 아니에요 \n']
_
질문:  ['남편이 \n']
정답:  ['\t 엄청 \n']
예상답:  ['힘든 시간이죠 \n']
_
질문:  ['지금이라도 중국어 공부할까 \n']
정답:  ['\t 나중에 도움이 될 거예요 \n']
예상답:  ['좋은 사람 만날 거예요 \n']
_
질문:  ['여친이 있는 사람인데 왜 나한테 \n']
정답:  ['\t 수도 있어요 \n']
예상답:  ['마음이 편하길 바랍니다 \n']
_
질문:  ['남동생한테 자꾸 화내게 되네 \n']
정답:  ['\t 화를 참는 연습을 해보세요 \n']
예상답:  ['화를 참는 연습을 해보세요 \n']
_
질문:  ['\n']
정답:  ['\t 잠은 정말 \n']
예상답:  ['추억도 시간도 지나가요 \n']
_
질문:  ['같은데 \n']
정답:  ['\t 당연하죠 \n']
예상답:  ['식초 넣고 빨래해보세요 \n']
_
질문:  ['남자친구가 인기가 거의 \n']
정답:  ['\t 멋진 사람이랑 좋겠네요 \n']
예상답:  ['내 집 마련 축하드려요 \n']

In [0]:
dataset_file = path + '/chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

In [0]:
for i in range(10) :
  print(test_a_seq[i])
  print(test_a[i])
  print(tokenizer.sequences_to_texts([test_a_seq[i]]))


[2, 156, 249, 15, 159, 1]
	 다시 새로 사는 게 마음 편해요. 

['\t 다시 사는 게 마음 \n']
[2, 156, 249, 15, 159, 1]
	 다시 새로 사는 게 마음 편해요. 

['\t 다시 사는 게 마음 \n']
[2, 8, 17, 59, 24, 1]
	 잘 모르고 있을 수도 있어요. 

['\t 잘 있을 수도 있어요 \n']
[2, 683, 22, 1]
	 시간을 정하고 해보세요. 

['\t 시간을 해보세요 \n']
[2, 683, 22, 1]
	 시간을 정하고 해보세요. 

['\t 시간을 해보세요 \n']
[2, 1]
	 혼자를 즐기세요. 

['\t \n']
[2, 681, 12, 1]
	 다음 달에는 더 절약해봐요. 

['\t 다음 더 \n']
[2, 1182, 469, 1183, 350, 1184, 220, 1185, 221, 350, 587, 351, 24, 1]
	 저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요 

['\t 저를 만들어 준 사람을 부모님 저랑 이야기해 주는 사람을 친구로 생각하고 있어요 \n']
[2, 506, 1]
	 휴식도 필요하죠. 

['\t 필요하죠 \n']
[2, 76, 35, 1]
	 맛있게 드세요. 

['\t 맛있게 드세요 \n']


In [0]:
for test_seq, test_labels in test_ds:

    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    print('_')
    print('질문: ', test_text)
    print('정답: ', gt_text)
    

_
질문:  ['\n']
정답:  ['\t 다시 사는 게 마음 \n']
_
질문:  ['안돼 \n']
정답:  ['\t 다시 사는 게 마음 \n']
_
질문:  ['왜 \n']
정답:  ['\t 잘 있을 수도 있어요 \n']
_
질문:  ['거 매일 하는 중 \n']
정답:  ['\t 시간을 해보세요 \n']
_
질문:  ['자꾸 \n']
정답:  ['\t 시간을 해보세요 \n']
_
질문:  ['좋다 \n']
정답:  ['\t \n']
_
질문:  ['가스비 \n']
정답:  ['\t 다음 더 \n']
_
질문:  ['가족이 누구야 \n']
정답:  ['\t 저를 만들어 준 사람을 부모님 저랑 이야기해 주는 사람을 친구로 생각하고 있어요 \n']
_
질문:  ['간만에 중 \n']
정답:  ['\t 필요하죠 \n']
_
질문:  ['\n']
정답:  ['\t 맛있게 드세요 \n']
_
질문:  ['갈까 말까 \n']
정답:  ['\t 가세요 \n']
_
질문:  ['감기 같애 \n']
정답:  ['\t \n']
_
질문:  ['없어 \n']
정답:  ['\t \n']
_
질문:  ['같이 친구가 없어 \n']
정답:  ['\t 혼자도 좋아요 \n']
_
질문:  ['같이 나만 거 같아 \n']
정답:  ['\t 잘 않을 거예요 \n']
_
질문:  ['같이 할 수 있는 뭐 있을까 \n']
정답:  ['\t 서로를 더 많이 될 거예요 \n']
_
질문:  ['\n']
정답:  ['\t \n']
_
질문:  ['\n']
정답:  ['\t 방학이 참 \n']
_
질문:  ['\n']
정답:  ['\t 게 좋겠네요 \n']
_
질문:  ['\n']
정답:  ['\t 게 좋겠네요 \n']
_
질문:  ['없어 \n']
정답:  ['\t 그게 제일 중요한 \n']
_
질문:  ['개인적인 다 시켜 \n']
정답:  ['\t 하세요 \n']
_
질문:  ['\n']
정답:  ['\t 잠깐 자도 괜찮아요 \n']
_
질문:  ['\n']
정답:  ['\t 저도 \n']
_

In [0]:
test_labels.numpy()

array([[   2, 1391,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [0]:
test_labels[2]

<tf.Tensor: shape=(), dtype=int32, numpy=1391>